In [59]:
import torch
from torch import nn
from d2l import torch as d2l

## **首先NiN网的思路非常有趣。在像alex和vgg等网络中，我们将最后卷积层的输出与全连接层进行连接，这会产生非常多的参数，参数太多也是导致过拟合的关键。NiN网络可以解决这样的问题。在一个nin块中，我们往往使用两个1*1的卷积层对每个通道上的像素进行一个提取，这样卷积完之后输出的每个特征点就是输入的每一个通道上的对应位置的像素的信息。之后，经过多个NiN块之后，最关键的一点来了，我们将最后输出的特征维度进行自适应平均到 1 * 1 ,之后reshape到二维，将输出的通道数视为不同的特征。**
* **NiN块由一个普通的卷积层和两个1*1核大小的卷积层构成。普通的卷积层进行不同特征的提取，1 * 1的卷积层进行不同通道的信息集合，之后输入激活函数进行非线性输出**

In [60]:
batch_size = 128
train_iter , test_iter = d2l.load_data_fashion_mnist( batch_size ,resize= 224 )

In [61]:
def NiN_block( input_channels   , out_channels ,  kernel_size ,  strides ,  padding ):
    return( nn.Sequential(  
                            nn.Conv2d( input_channels , out_channels , kernel_size , strides , padding ),
                            nn.ReLU(),
                            nn.Conv2d( out_channels , out_channels ,kernel_size = 1 ), nn.ReLU(),
                            nn.Conv2d( out_channels , out_channels , kernel_size = 1 ),  nn.ReLU()

      )
      )

In [62]:
net = nn.Sequential( 
    NiN_block( 1,96,kernel_size= 11,strides= 4,padding=0 ),
    nn.MaxPool2d( kernel_size=3 , stride = 2 ),
    NiN_block( 96 , 256 , kernel_size=5,  strides=1 ,padding= 2 ),
    nn.MaxPool2d( kernel_size=3 , stride=2  ),
    NiN_block( 256 , 384 , kernel_size=3 , strides=1, padding= 1 ),
    nn.MaxPool2d( kernel_size=3 , stride=2 ),
    nn.Dropout( 0.5 ),
    NiN_block( 384 , 10 , kernel_size=3 , strides=1, padding=1),
    nn.AdaptiveAvgPool2d( (1,1)),#将输出的特征输出为1*1
    nn.Flatten()#之后展开为Batchsize*10
)

In [63]:
x = torch.rand( size = (1 , 1, 224 , 244 ))
for layer in net:
    x = layer(x)
    print( layer.__class__.__name__ , 'output_size:' , x.shape )

Sequential output_size: torch.Size([1, 96, 54, 59])
MaxPool2d output_size: torch.Size([1, 96, 26, 29])
Sequential output_size: torch.Size([1, 256, 26, 29])
MaxPool2d output_size: torch.Size([1, 256, 12, 14])
Sequential output_size: torch.Size([1, 384, 12, 14])
MaxPool2d output_size: torch.Size([1, 384, 5, 6])
Dropout output_size: torch.Size([1, 384, 5, 6])
Sequential output_size: torch.Size([1, 10, 5, 6])
AdaptiveAvgPool2d output_size: torch.Size([1, 10, 1, 1])
Flatten output_size: torch.Size([1, 10])


In [68]:
len(train_iter) , len(train_iter)//5

(469, 93)

In [70]:
def train_simple_NiN( net , train_iter ,  test_iter , num_epochs , lr , device ):
    print('training on ', device )
    net.to( device )
    loss = nn.CrossEntropyLoss()
    optimzer = torch.optim.Adam( net.parameters() , lr )
    num_batch = len( train_iter )
    for epoch in range( num_epochs ):
        #存储多个批次的loss,准确个数，和总样本数
        metrics = d2l.Accumulator(3)
        for i , ( X,y) in enumerate( train_iter ):
            X = X.to( device )
            y = y.to( device )
            optimzer.zero_grad()
            y_hat = net( X )
            l = loss( y_hat , y )
            l.backward()
            optimzer.step()
            with torch.no_grad():
                metrics.add( l*X.shape[0] , d2l.accuracy( y_hat , y ) ,  y.numel() )
            train_loss = metrics[0]/metrics[2]
            train_acc  = metrics[1]/metrics[2]
            if (i+1) % (num_batch // 3 ) == 0 or i == num_batch-1:

                print( f'num_batch:{i+1} , avg_loss:{train_loss:.2f} , avg_accuracy:{train_acc:.2f}' )
        test_acc  = d2l.evaluate_accuracy_gpu( net , test_iter , device )
        print( f'epochs:{epoch} ,test_acc:{test_acc:.2f} ')
        



In [71]:
self_lr , self_num_epochs = 0.001 , 10
train_simple_NiN( net , train_iter , test_iter , num_epochs=self_num_epochs ,lr=self_lr , device=d2l.try_gpu() )

training on  cuda:0
num_batch:93 , avg_loss:2.19 , avg_accuracy:0.17
num_batch:186 , avg_loss:1.93 , avg_accuracy:0.29
num_batch:279 , avg_loss:1.70 , avg_accuracy:0.38
num_batch:372 , avg_loss:1.54 , avg_accuracy:0.43
num_batch:465 , avg_loss:1.42 , avg_accuracy:0.48
num_batch:469 , avg_loss:1.42 , avg_accuracy:0.48
epochs:0 ,test_acc:0.71 
num_batch:93 , avg_loss:0.81 , avg_accuracy:0.71
num_batch:186 , avg_loss:0.78 , avg_accuracy:0.72
num_batch:279 , avg_loss:0.75 , avg_accuracy:0.73
num_batch:372 , avg_loss:0.74 , avg_accuracy:0.74
num_batch:465 , avg_loss:0.72 , avg_accuracy:0.74
num_batch:469 , avg_loss:0.72 , avg_accuracy:0.74
epochs:1 ,test_acc:0.78 
num_batch:93 , avg_loss:0.62 , avg_accuracy:0.77
num_batch:186 , avg_loss:0.61 , avg_accuracy:0.78
num_batch:279 , avg_loss:0.60 , avg_accuracy:0.79
num_batch:372 , avg_loss:0.59 , avg_accuracy:0.79
num_batch:465 , avg_loss:0.58 , avg_accuracy:0.79
num_batch:469 , avg_loss:0.58 , avg_accuracy:0.79
epochs:2 ,test_acc:0.81 
num_batc

KeyboardInterrupt: 

In [ ]:
lr , num_epochs = 0.001 , 10
d2l.train_ch6( net , train_iter ,test_iter , num_epochs , lr , d2l.try_gpu() )